<a href="https://colab.research.google.com/github/torresguilherme/Sistema-De-Doa-es/blob/main/TrabalhoA2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Grupo Quinta feira (Cadastro de doações)

• Guilherme José Silva Torres - 38141264
• Cauã da Costa Cordeiro - 38446880
• Pedro Henrique Pontes - 39294072
• Arthur Moreira do Vale - 38634384
• Johnatan Antunes- 39333086
• Andre Felipe Oliveira - 38536935

💻 O que é o sistema?

Um painel interativo desenvolvido em Python com Google Colab, que permite:
	•	Cadastrar alimentos e controlar o estoque com datas de validade
	•	Cadastrar ONGs que recebem doações
	•	Registrar doações associando alimento + quantidade + ONG + doador
	•	Visualizar relatórios com alertas de vencimento e estoque
	•	Exportar tudo para Excel, de forma rápida e organizada

⸻

🧩 Tecnologias utilizadas
	•	Python – Linguagem principal
	•	SQLite – Banco de dados leve e embutido
	•	pandas – Manipulação de dados e exportação
	•	ipywidgets – Interface visual e interativa dentro do Google Colab
	•	Google Colab – Ambiente gratuito e online de execução

⸻

🗂️ Como o sistema está organizado

🔸 1. Cadastro de alimentos
	•	Nome, quantidade, unidade (kg, g, L…), validade
	•	Ao cadastrar, os alimentos entram no banco e aparecem no menu de doações

🔸 2. Cadastro de ONGs
	•	Nome e contato da ONG
	•	As ONGs aparecem como destino no momento da doação

🔸 3. Registro de doações
	•	Usuário informa o doador, escolhe o alimento e a ONG
	•	O sistema verifica o estoque antes de confirmar
	•	Após confirmar, a quantidade é abatida automaticamente do estoque

🔸 4. Listagens inteligentes
	•	Alimentos listados com status:
	•	✅ OK
	•	⚠️ Próximo da validade ou estoque baixo
	•	❌ Vencido
	•	ONGs e doações podem ser visualizadas com dados completos

🔸 5. Exportação de relatórios
	•	Gera 3 arquivos .xlsx com os dados de:
	•	Alimentos
	•	ONGs
	•	Doações

⸻

💡 Destaques do sistema
	•	Interface 100% interativa e intuitiva
	•	Indicadores visuais de validade e estoque
	•	Atualização automática dos menus suspensos
	•	Códigos limpos, comentados e organizados em funções reutilizáveis
	•	Pode ser facilmente adaptado para qualquer tipo de controle de estoque e repasse (roupas, medicamentos, etc.)

In [ ]:
import sqlite3  # Conecta com o banco de dados SQLite (leve e fácil de usar)
from datetime import datetime  # Para trabalhar com datas e horários
import ipywidgets as widgets  # Cria elementos interativos (caixas de texto, botões, etc.)
from IPython.display import display, clear_output  # Exibe e limpa a interface no Google Colab
import pandas as pd  # Para lidar com tabelas e criar relatórios em Excel

In [ ]:
# --- CONFIGURAÇÕES INICIAIS DO NOSSO SISTEMA ---
NOME_BANCO_DADOS = 'gestao_doacoes.db'  # Nome do nosso arquivo de banco de dados
DATE_FORMAT = "%Y-%m-%d"  # Formato padrão para datas (ex: 2025-06-01)
DATETIME_FORMAT = "%Y-%m-%d %H:%M:%S"  # Formato padrão para data e hora completas

In [ ]:
# --- CONECTANDO COM O BANCO DE DADOS ---
conn = sqlite3.connect(NOME_BANCO_DADOS)  # Conecta ao arquivo do banco de dados (cria se não existir)
cursor = conn.cursor()  # Cursor permite enviar comandos SQL para o banco

In [ ]:
# --- CRIANDO AS TABELAS, CASO AINDA NÃO EXISTAM ---
# Tabela de ONGs cadastradas
cursor.execute('''
    CREATE TABLE IF NOT EXISTS ongs (
        id INTEGER PRIMARY KEY AUTOINCREMENT,
        nome TEXT NOT NULL,
        contato TEXT
    )
''')

# Tabela de alimentos disponíveis
cursor.execute('''
    CREATE TABLE IF NOT EXISTS alimentos (
        id INTEGER PRIMARY KEY AUTOINCREMENT,
        nome TEXT NOT NULL,
        quantidade INTEGER NOT NULL,
        unidade TEXT NOT NULL,
        validade TEXT NOT NULL
    )
''')

# Tabela que registra cada doação feita
cursor.execute('''
    CREATE TABLE IF NOT EXISTS doacoes (
        id INTEGER PRIMARY KEY AUTOINCREMENT,
        doador TEXT NOT NULL,
        alimento_id INTEGER NOT NULL,
        quantidade INTEGER NOT NULL,
        ong_id INTEGER NOT NULL,
        data_doacao TEXT NOT NULL,
        FOREIGN KEY (alimento_id) REFERENCES alimentos(id),
        FOREIGN KEY (ong_id) REFERENCES ongs(id)
    )
''')

conn.commit()  # Salva todas as alterações feitas até aqui no banco

In [ ]:
# --- UNIDADES PADRÃO PARA EVITAR ERROS DE DIGITAÇÃO ---
UNIDADES_FIXAS = ['kg', 'g', 'L', 'unidade', 'pacote']

# --- CAMPOS PARA CADASTRAR ALIMENTOS ---
nome_alimento = widgets.Text(description='Nome:')
quantidade_alimento = widgets.IntText(description='Qtd:')
validade_alimento = widgets.DatePicker(description='Validade:')
unidade_dropdown = widgets.Dropdown(description='Unidade:', options=UNIDADES_FIXAS)
botao_cadastrar_alimento = widgets.Button(description='Cadastrar Alimento')

# --- CAMPOS PARA CADASTRAR ONGS ---
nome_ong = widgets.Text(description='Nome:')
contato_ong = widgets.Text(description='Contato:')
botao_cadastrar_ong = widgets.Button(description='Cadastrar ONG')

# --- CAMPOS PARA REGISTRAR UMA NOVA DOAÇÃO ---
doador_input = widgets.Text(description='Doador:')
alimento_doacao_dropdown = widgets.Dropdown(description='Alimento:', options=[])
quantidade_doacao = widgets.IntText(description='Qtd:')
ong_doacao_dropdown = widgets.Dropdown(description='ONG:', options=[])
botao_cadastrar_doacao = widgets.Button(description='Registrar Doação')

# --- BOTÃO PARA EXPORTAR OS RELATÓRIOS EM EXCEL ---
botao_exportar = widgets.Button(description='Exportar Relatórios')

In [ ]:
# --- FUNÇÃO PARA ATUALIZAR O DROPDOWN DE ALIMENTOS ---
def atualizar_dropdown_alimentos():
    cursor.execute('SELECT id, nome FROM alimentos')
    alimentos = [(f"{a[0]} - {a[1]}", a[0]) for a in cursor.fetchall()]
    alimento_doacao_dropdown.options = alimentos

# --- FUNÇÃO PARA ATUALIZAR O DROPDOWN DE ONGS ---
def atualizar_dropdown_ongs():
    cursor.execute('SELECT id, nome FROM ongs')
    ongs = [(f"{o[0]} - {o[1]}", o[0]) for o in cursor.fetchall()]
    ong_doacao_dropdown.options = ongs

# --- AQUI COMEÇAM AS AÇÕES DO SISTEMA ---
# Cadastrar um novo alimento

def cadastrar_alimento(b):
    with container:
        clear_output()
        nome = nome_alimento.value.strip()
        quantidade = quantidade_alimento.value
        validade = validade_alimento.value
        unidade = unidade_dropdown.value

        # Verifica se todos os campos foram preenchidos corretamente
        if not (nome and quantidade > 0 and validade and unidade):
            print("❌ Por favor, preencha todos os campos corretamente.")
            return

        validade_str = validade.strftime(DATE_FORMAT)
        cursor.execute('''
            INSERT INTO alimentos (nome, quantidade, unidade, validade)
            VALUES (?, ?, ?, ?)
        ''', (nome, quantidade, unidade, validade_str))
        conn.commit()
        print(f"✅ Alimento '{nome}' cadastrado com sucesso!")
        atualizar_dropdown_alimentos()

# Listar os alimentos já cadastrados com status visual

def listar_alimentos():
    with container:
        clear_output()
        df = pd.read_sql_query('SELECT * FROM alimentos', conn)

        if df.empty:
            print("❌ Nenhum alimento foi cadastrado ainda.")
            return

        hoje = datetime.now().date()
        status = []

        for _, row in df.iterrows():
            validade = datetime.strptime(row['validade'], DATE_FORMAT).date()
            dias = (validade - hoje).days
            if dias < 0:
                s = '❌ VENCIDO'
            elif dias <= 30:
                s = '⚠️ VENCE LOGO'
            else:
                s = '✅ OK'
            status.append(s)

        df['Status'] = status
        df['Estoque'] = df['quantidade'].apply(lambda x: '⚠️ Baixo' if x <= 5 else '✅ OK')
        display(df)

# Cadastrar nova ONG

def cadastrar_ong(b):
    with container:
        clear_output()
        nome = nome_ong.value.strip()
        contato = contato_ong.value.strip()

        if not nome:
            print("❌ O nome da ONG é obrigatório.")
            return

        cursor.execute('INSERT INTO ongs (nome, contato) VALUES (?, ?)', (nome, contato))
        conn.commit()
        print(f"✅ ONG '{nome}' cadastrada com sucesso!")
        atualizar_dropdown_ongs()

# Listar as ONGs cadastradas

def listar_ongs():
    with container:
        clear_output()
        df = pd.read_sql_query('SELECT * FROM ongs', conn)

        if df.empty:
            print("❌ Nenhuma ONG cadastrada ainda.")
        else:
            display(df)

# Registrar uma nova doação

def cadastrar_doacao(b):
    with container:
        clear_output()
        doador = doador_input.value.strip()
        alimento_id = alimento_doacao_dropdown.value
        quantidade = quantidade_doacao.value
        ong_id = ong_doacao_dropdown.value

        if not (doador and alimento_id and quantidade > 0 and ong_id):
            print("❌ Preencha todos os campos corretamente.")
            return

        cursor.execute('SELECT quantidade FROM alimentos WHERE id = ?', (alimento_id,))
        atual = cursor.fetchone()

        if not atual or atual[0] < quantidade:
            print("❌ Não há estoque suficiente desse alimento.")
            return

        data_doacao = datetime.now().strftime(DATETIME_FORMAT)
        cursor.execute('''
            INSERT INTO doacoes (doador, alimento_id, quantidade, ong_id, data_doacao)
            VALUES (?, ?, ?, ?, ?)
        ''', (doador, alimento_id, quantidade, ong_id, data_doacao))

        cursor.execute('UPDATE alimentos SET quantidade = quantidade - ? WHERE id = ?', (quantidade, alimento_id))
        conn.commit()
        print(f"✅ Doação registrada por '{doador}' com sucesso!")
        atualizar_dropdown_alimentos()

# Mostrar todas as doações realizadas

def listar_doacoes():
    with container:
        clear_output()
        df = pd.read_sql_query('''
            SELECT d.id, d.doador, a.nome as alimento, d.quantidade, o.nome as ONG, d.data_doacao
            FROM doacoes d
            JOIN alimentos a ON d.alimento_id = a.id
            JOIN ongs o ON d.ong_id = o.id
            ORDER BY d.data_doacao DESC
        ''', conn)

        if df.empty:
            print("❌ Nenhuma doação foi registrada ainda.")
        else:
            display(df)

# Gera relatórios em Excel com todos os dados cadastrados

def exportar_relatorios(b):
    with container:
        clear_output()

        alimentos = pd.read_sql_query('SELECT * FROM alimentos', conn)
        doacoes = pd.read_sql_query('''
            SELECT d.id, d.doador, a.nome as alimento, d.quantidade, o.nome as ONG, d.data_doacao
            FROM doacoes d
            JOIN alimentos a ON d.alimento_id = a.id
            JOIN ongs o ON d.ong_id = o.id
        ''', conn)
        ongs = pd.read_sql_query('SELECT * FROM ongs', conn)

        alimentos.to_excel('relatorio_alimentos.xlsx', index=False)
        doacoes.to_excel('relatorio_doacoes.xlsx', index=False)
        ongs.to_excel('relatorio_ongs.xlsx', index=False)

        print("✅ Relatórios exportados com sucesso!")

# --- VINCULA CADA BOTÃO À SUA FUNÇÃO ---
botao_cadastrar_alimento.on_click(cadastrar_alimento)
botao_cadastrar_ong.on_click(cadastrar_ong)
botao_cadastrar_doacao.on_click(cadastrar_doacao)
botao_exportar.on_click(exportar_relatorios)

# --- PREPARANDO A INTERFACE ---
atualizar_dropdown_alimentos()
atualizar_dropdown_ongs()
container = widgets.Output()
menu = widgets.ToggleButtons(
    options=[
        'Cadastrar Alimento',
        'Listar Alimentos',
        'Cadastrar ONG',
        'Listar ONGs',
        'Registrar Doação',
        'Listar Doações',
        'Exportar Relatórios'
    ],
    description='Menu:'
)

def exibir_menu(change):
    with container:
        clear_output()
        opcao = menu.value

        if opcao == 'Cadastrar Alimento':
            display(widgets.VBox([nome_alimento, quantidade_alimento, validade_alimento, unidade_dropdown, botao_cadastrar_alimento]))
        elif opcao == 'Listar Alimentos':
            listar_alimentos()
        elif opcao == 'Cadastrar ONG':
            display(widgets.VBox([nome_ong, contato_ong, botao_cadastrar_ong]))
        elif opcao == 'Listar ONGs':
            listar_ongs()
        elif opcao == 'Registrar Doação':
            display(widgets.VBox([doador_input, alimento_doacao_dropdown, quantidade_doacao, ong_doacao_dropdown, botao_cadastrar_doacao]))
        elif opcao == 'Listar Doações':
            listar_doacoes()
        elif opcao == 'Exportar Relatórios':
            exportar_relatorios(None)

menu.observe(exibir_menu, names='value')

# --- EXIBIR O MENU E ÁREA DE CONTEÚDO ---
display(menu, container)


ToggleButtons(description='Menu:', options=('Cadastrar Alimento', 'Listar Alimentos', 'Cadastrar ONG', 'Listar…

Output()